<a href="https://colab.research.google.com/github/mohit-work/advanced/blob/main/medical_chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multilingual Speech Enabled Healthcare Chatbot

## Step 1: Installing and importing the libraries

In [1]:
!pip install gTTS
from gtts import gTTS
!pip install googletrans==4.0.0-rc1
from googletrans import Translator

In [ ]:
!pip install SpeechRecognition
!pip install pydub
from IPython.display import display, Javascript, Audio
import numpy as np
import base64
import io
import ipywidgets as widgets
import subprocess
import speech_recognition as sr
import speech_recognition as sr
from pydub import AudioSegment

In [3]:
!pip uninstall accelerate peft bitsandbytes transformers trl -y
!pip install accelerate peft==0.13.2 bitsandbytes transformers trl==0.12.0

Found existing installation: accelerate 1.8.0
Uninstalling accelerate-1.8.0:
  Successfully uninstalled accelerate-1.8.0
Found existing installation: peft 0.13.2
Uninstalling peft-0.13.2:
  Successfully uninstalled peft-0.13.2
Found existing installation: bitsandbytes 0.46.0
Uninstalling bitsandbytes-0.46.0:
  Successfully uninstalled bitsandbytes-0.46.0
Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
Found existing installation: trl 0.12.0
Uninstalling trl-0.12.0:
  Successfully uninstalled trl-0.12.0
  Using cached accelerate-1.8.0-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.13.2-py3-none-any.whl.metadata (13 kB)
  Using cached bitsandbytes-0.46.0-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)
  Using cached trl-0.12.0-py3-none-any.whl.metadata (10 kB)
Using cached peft-0.13.2-py3-none-any.whl (320 kB)
Using cached trl

In [4]:
!pip install huggingface_hub

In [5]:
import torch
import re
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

## Step 2: Loading the model

In [6]:
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "abneraigc/llama2finetune_demo",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_compute_dtype = getattr(torch, "float16"), bnb_4bit_quant_type = "nf4"))
llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Step 3: Loading the tokenizer

In [7]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "abneraigc/llama2finetune_demo", trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

## Step 4: Setting the training arguments

In [8]:
training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size = 1,gradient_accumulation_steps=4, max_steps = 1,report_to="none", logging_strategy="no")

## Step 5: Creating the Supervised Fine-Tuning trainer

In [9]:
llama_sft_trainer = SFTTrainer(model = llama_model,
                               args = training_arguments,
                               train_dataset = load_dataset(path = "aboonaji/wiki_medical_terms_llam2_format", split = "train"),
                               tokenizer = llama_tokenizer,
                               peft_config = LoraConfig(task_type = "CAUSAL_LM", r = 64, lora_alpha = 16, lora_dropout = 0.1),
                               dataset_text_field = "text")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label

## Step 6: Training the model

In [10]:
import os
llama_sft_trainer.train()
# API key: f5d55cb4b339b0b2a6c49c9bb6f269e323bd030b

Step,Training Loss


TrainOutput(global_step=1, training_loss=1.9011927843093872, metrics={'train_runtime': 27.8374, 'train_samples_per_second': 0.144, 'train_steps_per_second': 0.036, 'total_flos': 116149306859520.0, 'train_loss': 1.9011927843093872, 'epoch': 0.0005830053927998834})

# Input type: Voice or Text

In [11]:
print('Select 1 for voice input')
print('Select 2 for manual input')
choice=int(input())
if choice!=1 and choice!=2:
  print('Invalid choice')

Select 1 for voice input
Select 2 for manual input
2


# Step 7: Language selection and speech recognition

In [12]:
print('Enter Language')
language=str(input()) # en-english,hi-hindi,te-telugu,ta-tamil,bn-bengali etc
# JavaScript code to record audio
if choice==1:
 def record_audio():
    display(Javascript('''
        async function recordAudio() {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            const mediaRecorder = new MediaRecorder(stream);
            const audioChunks = [];

            mediaRecorder.ondataavailable = event => {
                audioChunks.push(event.data);
            };

            mediaRecorder.start();

            const stopButton = document.createElement('button');
            stopButton.innerText = 'Stop Recording';
            document.body.appendChild(stopButton);

            stopButton.onclick = async () => {
                mediaRecorder.stop();
                stopButton.disabled = true;
            };

            mediaRecorder.onstop = async () => {
                const audioBlob = new Blob(audioChunks, { type: 'audio/webm' });
                const reader = new FileReader();
                reader.readAsDataURL(audioBlob);
                reader.onloadend = () => {
                    google.colab.kernel.invokeFunction('notebook.recorded_audio', [reader.result], {});
                };
            };
        }

        recordAudio();
    '''))

 # Function to save the recorded audio
 def save_audio(audio_data_url):
     audio_bytes = base64.b64decode(audio_data_url.split(',')[1])
     with open('recorded_audio.webm', 'wb') as f:
         f.write(audio_bytes)
     print("Audio saved successfully!")

 # Register the function to handle audio data
 from google.colab import output
 output.register_callback('notebook.recorded_audio', save_audio)

 # Start recording
 record_audio()

Enter Language
en


In [13]:
if choice==1:
# Step 1: Convert WebM to WAV
 def convert_webm_to_wav(input_file="recorded_audio.webm", output_file="recorded_audio.wav"):
    command = f"ffmpeg -i {input_file} -ar 16000 -ac 1 -c:a pcm_s16le {output_file} -y"
    subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("Conversion complete: WebM → WAV")

# Run the conversion and transcription
 convert_webm_to_wav()

In [14]:
if choice==1:
 # Initialize recognizer
 recognizer = sr.Recognizer()
 # Load the audio file
 with sr.AudioFile("recorded_audio.wav") as source:
     audio_data = recognizer.record(source)
  # Recognize speech in the specified language (e.g., Spanish)
  # Use appropriate language code for your language
 try:
    text1 = recognizer.recognize_google(audio_data, language=language)
 except sr.UnknownValueError:
     print("Could not understand the audio")
 except sr.RequestError:
     print("Could not request results from Google Speech Recognition service")

## Step 8: Chatting with the model

In [15]:
print('Enter Prompt')
if choice==2:
  text1 = str(input())
user_prompt = text1
if choice==1:
 print(user_prompt)
translator = Translator()
translated1 = translator.translate(user_prompt, dest='en')
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 500)
model_answer = text_generation_pipeline(f"[INST] {translated1.text} [/INST]")
text=model_answer[0]['generated_text']
cleaned_text = re.sub(r'\[INST\].*?\[/INST\]', '', text).strip()
translated2 = translator.translate(cleaned_text, dest=language)
translated_text=translated2.text
tts = gTTS(text=translated_text, lang=language)
audio_file = 'output_audio.mp3'
tts.save(audio_file)
from IPython.display import Audio
print(translated_text)
Audio(audio_file,autoplay=True)

Enter Prompt
headache in the morning


Device set to use cuda:0


A morning headache can be caused by a variety of factors, including:
nobody likes waking up with a headache, especially when it disrupts your daily routine. here are some possible causes of morning headaches and some tips to help you alleviate them:

1. Sleep Disturbances: Poor sleep quality or not getting enough sleep can lead to morning headaches. Make sure you're getting enough restful sleep each night and establishing a consistent sleep schedule.
2. Dehydration: Dehydration can cause headaches, especially if you didn't drink enough water during the day. Make sure to drink a glass of water as soon as you wake up to rehydrate.
3. Caffeine Withdrawal: If you consume caffeine in the afternoon or evening, you may experience a withdrawal headache in the morning. Try to limit your caffeine intake to the morning hours to avoid this.
4. Sinus Pressure: Sinus pressure and congestion can cause morning headaches. If you experience sinus pressure, try using saline nasal sprays or taking over-th